In [38]:
import random

from tqdm import tqdm
from datasets import load_from_disk, DatasetDict

In [39]:
wiki_datasets = load_from_disk("/opt/ml/data/wiki_preprocessed_droped")
train_dataset = load_from_disk("/opt/ml/data/generator_dataset")
wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")

In [40]:
train_dataset

Dataset({
    features: ['context', 'question', 'id', 'title', 'document_id', 'answers'],
    num_rows: 35740
})

In [41]:
dicts = []
for wiki in tqdm(wiki_datasets):
  wiki_dick = {}
  wiki_dick['text'] = wiki['text']
  wiki_dick['meta'] = {
    'title': wiki['title'],
    'document_id': wiki['document_id']
  }
  dicts.append(wiki_dick)

100%|██████████| 55962/55962 [00:09<00:00, 5889.42it/s]


In [42]:
dict_id = []
for wiki in tqdm(dicts):
  dict_id.append(wiki['meta']['document_id'])

100%|██████████| 55962/55962 [00:00<00:00, 1071289.36it/s]


In [43]:
error_context = []

def drop_not_doc(datasets):
  not_include = []
  for data in tqdm(datasets):
    if data['document_id'] not in dict_id:
      not_include.append(data['document_id'])
  def get_distance(a, b):
    return abs(a - b)
  def change_doc_id(example):
    # if example['document_id'] not in not_include:
    #   return example
    doc = wiki_datasets.get_nearest_examples("text", example['context'][:1024], k=1)
    document_id = doc[1]['document_id'][0]
    title = doc[1]['title'][0]
    context_list = doc[1]['text'][0].split(example['answers']['text'][0])
    if len(context_list) == 1:
      error_context.append(example, context_list)
      return example
    distance = len(example['context'])
    location = 0
    for context in context_list:
      location += len(context)
      new_distance = get_distance(location, example['answers']['answer_start'][0])
      if distance > new_distance:
        distance = new_distance
      else:
        location -= len(context)
        break
    example['answers']['answer_start'][0] = location
    example['document_id'] = document_id
    example['title'] = title
    example['context'] = example['answers']['text'][0].join(context_list)
    return example
  return datasets.map(change_doc_id)

In [30]:
new_train_dataset = drop_not_doc(train_dataset)

100%|██████████| 35740/35740 [00:20<00:00, 1716.99it/s]


  0%|          | 0/35740 [00:00<?, ?ex/s]

/home/ubuntu/workspace/mrc_venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [44]:
len(error_context)

0

In [45]:
search_error = new_train_dataset.filter(lambda example: example['context'][example['answers']['answer_start'][0]:example['answers']['answer_start'][0]+len(example['answers']['text'][0])] != example['answers']['text'][0])

Loading cached processed dataset at /opt/ml/data/new_train_dataset/train/cache-989c01d5f466607c.arrow


In [46]:
search_error

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 0
})

In [47]:
valid_dataset = load_from_disk('/opt/ml/data/new_train_dataset')

In [48]:
valid_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3351
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 841
    })
})

In [49]:
data = {
  'train': new_train_dataset,
  'validation': valid_dataset['validation']
}
train = DatasetDict(data)
train.save_to_disk('new_gen_dataset') # 저장위치